# Generate new proteins using pre-trained genie

In [1]:
import os
import sys
import torch
import argparse
import numpy as np
from tqdm import tqdm, trange

gpu = input("GPU devices to use:")
if gpu == '':
    gpu = None
outdir = input("Save generated coordinates to:")
if outdir == '':
    outdir = 'outputs'
num_batches = input("number of batches:")
if num_batches == '':
    num_batches = 2
else:
    num_batches = int(num_batches)
batch_size = input("batch size:")
if batch_size == '':
    batch_size = 5
else:
    batch_size = int(batch_size)

if not os.path.exists(outdir):
    os.mkdir(outdir)

GPU devices to use: 0
Save generated coordinates to: 
number of batches: 1
batch size: 1


In [2]:
from genie.config import Config
from genie.diffusion.genie import Genie

config = Config('weights/configuration')
model = Genie.load_from_checkpoint('weights/genie_l_128_epoch=49999.ckpt', config=config)

device = 'cuda:{}'.format(gpu) if gpu is not None else 'cpu'
model = model.to(device)

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file weights/genie_l_128_epoch=49999.ckpt`


In [6]:
# sanity check
# min_length = 50
max_length = 128
max_n_res = model.config.io['max_n_res']
assert max_length <= max_n_res

# sample
# for length in trange(min_length, max_length + 1):
length = 128
for batch_idx in range(num_batches):
    mask = torch.cat([
        torch.ones((batch_size, length)),
        torch.zeros((batch_size, max_n_res - length))
    ], dim=1).to(device)
    ts = model.p_sample_loop(mask, verbose=False)[-1]
    for batch_sample_idx in range(ts.shape[0]):
        sample_idx = batch_idx * batch_size + batch_sample_idx
        coords = ts[batch_sample_idx].trans.detach().cpu().numpy()
        coords = coords[:length]
        np.savetxt(os.path.join(outdir, f'{length}_{sample_idx}.npy'), coords, fmt='%.3f', delimiter=',')

torch.Size([1, 128])


In [ ]:
save_path = f'{sample_dir}/len_{num_res_sample}_{b_idx}.pdb'
au.write_prot_to_pdb(sampled_diffusion[b_idx][-1], save_path, no_indexing=True)

In [31]:
import plotly.graph_objects as go
def create_scatter(pos_3d,
                   mode='markers',
                   marker_size=None,
                   name=None,
                   opacity=None,
                   color=None,
                   colorscale=None,
                   ):
    """Creates Scatter3D objects for use in plotly.
    Args:
        pos_3d: [N, 3] array containing N points with
            euclidean coordinates.
        mode: How to display points.
            Use 'markers' for scatter.
            Use 'lines' for lines connecting consecutive points.
            Use 'lines+markers' for scatter and lines.
        marker_size: Size of markers.
        name: Label of plotting layer to be displayed in legend.
        opacity: Transparency of points.
    """
    x, y, z = np.split(pos_3d, 3, axis=-1)
    args_dict = {
        'x': x[:, 0],
        'y': y[:, 0],
        'z': z[:, 0],
        'mode': mode,
        'marker': {}
    }
    if marker_size:
        args_dict['marker']['size'] = marker_size
    if name:
        args_dict['name'] = name
    if opacity:
        args_dict['opacity'] = opacity
    if color:
        args_dict['marker']['color'] = color
    if colorscale:
        args_dict['marker']['colorscale'] = colorscale
    return go.Scatter3d(**args_dict)

In [52]:
from plotly.subplots import make_subplots
%matplotlib inline
## Plot samples
num_res = np.sum(mask.cpu().numpy(), axis=-1)
nrows = int(np.sqrt(batch_size))
ncols = nrows
fig = make_subplots(
    rows=nrows, cols=ncols,
    specs=[[{'type': 'surface'}] * nrows]*ncols)
# Take last time step
# last_sample = coords
fig.update_layout(
    title_text=f'Samples',
    height=1000,
    width=1000,
)

sample_bb_3d = create_scatter(
    coords, mode='lines+markers', marker_size=3,
    opacity=1.0, name=f'Sample {1}: length={num_res[0]}')
fig.add_trace(sample_bb_3d, row=1, col=1)
        
fig.show(renderer='browser')

self-consistency to PDMLM and alphafold

In [57]:
path_to_PDB="/mnt/d/RESEARCH/DMREF/genie/outputs/128_0.npy"
output_dir="outputs/"

!python /mnt/d/RESEARCH/ProteinMPNN/protein_mpnn_run.py \
        --pdb_path $path_to_PDB \
        --out_folder $output_dir \
        --num_seq_per_target 2 \
        --sampling_temp "0.1" \
        --seed 37 \
        --batch_size 1

----------------------------------------
chain_id_jsonl is NOT loaded
----------------------------------------
fixed_positions_jsonl is NOT loaded
----------------------------------------
pssm_jsonl is NOT loaded
----------------------------------------
omit_AA_jsonl is NOT loaded
----------------------------------------
bias_AA_jsonl is NOT loaded
----------------------------------------
tied_positions_jsonl is NOT loaded
----------------------------------------
bias by residue dictionary is not loaded, or not provided
----------------------------------------
----------------------------------------
Number of edges: 48
Training noise level: 0.2A
Traceback (most recent call last):
  File "/mnt/d/RESEARCH/ProteinMPNN/protein_mpnn_run.py", line 423, in <module>
    main(args)   
  File "/mnt/d/RESEARCH/ProteinMPNN/protein_mpnn_run.py", line 218, in main
    X, S, mask, lengths, chain_M, chain_encoding_all, chain_list_list, visible_list_list, masked_list_list, masked_chain_length_list_lis

# Train dhaA dataset

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('-g', '--gpus', type=str, help='GPU devices to use')
parser.add_argument('-c', '--config', type=str, help='Path for configuration file', required=True)
args = parser.parse_args()